# Settings

In [1]:
import openai
from openai import OpenAI

import os
from dotenv import load_dotenv # API KEY 정보를 불러옵니다
load_dotenv('./credentials/.env') 
openai.api_key = os.environ.get("OPENAI_API_KEY")
api_key = openai.api_key

import json
import time


# OpenAI API를 사용하기 위한 클라이언트 객체를 생성합니다.
client = OpenAI(api_key=api_key)

# 수학 과외 선생님 역할을 하는 챗봇을 생성합니다.
# 이 챗봇은 간단한 문장이나 한 문장으로 질문에 답변합니다.
assistant = client.beta.assistants.create(
    name="Math Tutor",
    instructions="You are a personal math tutor. Answer questions briefly, in a sentence or less.",
    model="gpt-4-turbo-preview",
)


def show_json(obj):
    # obj의 모델을 JSON 형태로 변환한 후 출력합니다.
    display(json.loads(obj.model_dump_json()))

# 생성된 챗봇의 정보를 JSON 형태로 출력합니다.
show_json(assistant)

{'id': 'asst_6u3ztejiHTk52XcalB8J29oT',
 'created_at': 1727972526,
 'description': None,
 'instructions': 'You are a personal math tutor. Answer questions briefly, in a sentence or less.',
 'metadata': {},
 'model': 'gpt-4-turbo-preview',
 'name': 'Math Tutor',
 'object': 'assistant',
 'tools': [],
 'response_format': 'auto',
 'temperature': 1.0,
 'tool_resources': {'code_interpreter': None, 'file_search': None},
 'top_p': 1.0}

In [2]:
ASSISTANT_ID = assistant.id
print(f"[생성한 Assistants ID]\n{ASSISTANT_ID}")

[생성한 Assistants ID]
asst_6u3ztejiHTk52XcalB8J29oT


# Few-Shot

In [4]:
# 반복문에서 대기하는 함수
def wait_on_run(run, thread_id):
    while run.status == "queued" or run.status == "in_progress":
        # 3-3. 실행 상태를 최신 정보로 업데이트합니다.
        run = client.beta.threads.runs.retrieve(
            thread_id=thread_id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run


def submit_message(assistant_id, thread_id, user_message):
    # 3-1. 스레드에 종속된 메시지를 '추가' 합니다.
    client.beta.threads.messages.create(
        thread_id=thread_id, role="user", content=user_message
    )
    # 3-2. 스레드를 실행합니다.
    run = client.beta.threads.runs.create(
        thread_id=thread_id,
        assistant_id=assistant_id,
    )
    return run


def get_response(thread_id):
    # 3-4. 스레드에 종속된 메시지를 '조회' 합니다.
    return client.beta.threads.messages.list(thread_id=thread_id, order="asc")


def print_message(response):
    for res in response:
        print(f"[{res.role.upper()}]\n{res.content[0].text.value}\n")


def ask(assistant_id, thread_id, user_message):
    run = submit_message(
        assistant_id,
        thread_id,
        user_message,
    )
    # 실행이 완료될 때까지 대기합니다.
    run = wait_on_run(run, thread_id)
    print_message(get_response(thread_id).data[-2:]) 
        # 대화에서 최신 메세지 두개를 출력
            # data 리스트의 마지막 두 개의 항목을 가져오는 것을 의미
            # 즉, data 리스트의 끝부분에 있는 두 개의 메시지만 가져오고 있는 것
    return run

def create_new_thread():
    # 새로운 스레드를 생성합니다.
    thread = client.beta.threads.create()
    return thread


# thread_id = "기존 스레드 ID를 입력해 주세요"
thread_id = create_new_thread().id  # 새로운 스레드를 생성합니다.

user_message = """
다음 영어 문장을 한국어로 번역하세요:

영어: How are you?
한국어: 잘 지내세요?

영어: What's your name?
한국어: 당신의 이름은 무엇입니까?

영어: Where is the bathroom?
한국어:

"""

run = ask(ASSISTANT_ID, thread_id, user_message)

[USER]

다음 영어 문장을 한국어로 번역하세요:

영어: How are you?
한국어: 잘 지내세요?

영어: What's your name?
한국어: 당신의 이름은 무엇입니까?

영어: Where is the bathroom?
한국어:



[ASSISTANT]
화장실은 어디에 있습니까?



# Zero-shot Chain-of-Thought (CoT)

In [5]:
user_message = """
질문: 사과가 5개 있습니다. 2개를 더 샀습니다. 그 후 1개의 사과를 먹었습니다. 사과는 몇 개 남았나요?

단계적으로 생각해 봅시다:

답:
"""

run = ask(ASSISTANT_ID, thread_id, user_message)

[USER]

질문: 사과가 5개 있습니다. 2개를 더 샀습니다. 그 후 1개의 사과를 먹었습니다. 사과는 몇 개 남았나요?

단계적으로 생각해 봅시다:

답:


[ASSISTANT]
처음에 5개의 사과가 있습니다.
2개를 더 사면 사과는 5 + 2 = 7개가 됩니다.
1개의 사과를 먹으면 7 - 1 = 6개가 남습니다.

답: 6개



In [6]:
user_message = """
질문: 상자에 연필이 12자루 있습니다. 4자루를 꺼냈습니다. 연필은 몇 자루 남았나요?

단계적으로 생각해 봅시다:

답:
"""

run = ask(ASSISTANT_ID, thread_id, user_message)

[USER]

질문: 상자에 연필이 12자루 있습니다. 4자루를 꺼냈습니다. 연필은 몇 자루 남았나요?

단계적으로 생각해 봅시다:

답:


[ASSISTANT]
처음에 12자루의 연필이 있습니다.
4자루를 꺼내면 연필은 12 - 4 = 8자루가 남습니다.

답: 8자루

